# Amazon ML Hack

Team: rokos-basilisk

In [ ]:
# init and imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 1. Dataset

In [ ]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip

--2021-07-30 16:56:28--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.124.234
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.124.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip.1’

dataset52a7b21.zip. 100%[===================>]   1012M  12.4MB/s    in 86s     

2021-07-30 16:57:56 (11.7 MB/s) - ‘dataset52a7b21.zip.1’ saved [1061576029/1061576029]



In [ ]:
!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
replace dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## 1.1 Loading the data

In [ ]:

full_df = pd.read_csv('dataset/train.csv', escapechar = "\\", quoting = csv.QUOTE_NONE, encoding='utf-8')
df = full_df.sample(5000)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.fillna("", inplace=True)

## NLP Experiments

In [ ]:
df

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
844289,Di Grazia Women’s Butterfly Backpack Handbag (...,Di Grazia Handbags is an exclusive line of Pre...,"[Material - PU Leather,Dimension - 28*24*16 cm...",Di Grazia,19247
1780577,Levi's Men's Slim Fit Casual Trousers (84416-0...,,"[Color: Black,Material: Cotton,Pattern: Plain,...",Levi's,1017
2229499,Humor Gang Printed Back Cover for Huawei Nova ...,Super durable and amazing looking phone Cases ...,[Stand Out in Crowd with Exclusively Designed ...,Humor Gang,1045
1552849,Provizon OBJECTS21 Super King Size Double Bed ...,Mosquito net gives complete protection from mo...,"[Color - White,Size - 7x7 ft,King Size Double ...",Provizon,677
31497,Second May Handmade Brown Colour Leather Diary...,Second May Bound handmade paper with Genuine L...,"[Overall Length: 8 Inches, Width: 6.5 inch; Co...",Second May,1196
...,...,...,...,...,...
1076135,NOVICZ 12 Sets 100 ml. Kitchen Food Container ...,,[Transparent design to identify the content in...,NOVICZ,840
1306181,U.S. Polo Assn. Girls Straight Red Casual Dres...,U.S. Polo Assn. Girls Striped Cotton Straight ...,"[Pattern Type :: Striped,Fit Type :: Straight,...",US Polo Association,1687
494731,"Nourish Organics Brown Rice Cookies, 150g",,"[Healthy eating for Urban living,Certified Org...",Nourish Organics,103877
2017811,"Rv Enterprises Plastic Bag Sealing Clips, 18-P...",A set of 18 pieces (3 size x 6pc each) bag cli...,[The Sealing Clip Is Great For Keeping Leftove...,RV Enterprises,13966


In [ ]:

def clean_text(text):
    # split into words
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)

    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))

    from nltk.stem.porter import PorterStemmer
    porter = PorterStemmer()
    stop_words = list(stop_words)
    stop_words.extend(['use', 'etc', 'also', 'every', 'made'])
    stemmed = [porter.stem(word) for word in stop_words]

    words = set([w for w in words if not w in stop_words])

    return words


In [ ]:
df['description_keywords'] = df['DESCRIPTION'].apply(lambda x:clean_text(x))
df['title_keywords'] = df['TITLE'].apply(lambda x:clean_text(x))

In [ ]:
print(df['title_keywords'].iloc[0])
print(df['BRAND'].iloc[0])

print(df['title_keywords'].iloc[1])
print(df['BRAND'].iloc[1])

# Aula-trainer
# Aula-optical
# Johy-trainer
# Lolman-trainer

# df.groupby(['BRAND'])['BRAND'].apply(list)

# groupby trainer

{'women', 'handbag', 'butterfly', 'backpack', 'di', 'white', 'grazia', 'whiteflowerssmallbackpack'}
Di Grazia
{'men', 'trousers', 'levi', 'slim', 'x', 'casual', 'fit'}
Levi's


In [ ]:
for row in df.iloc:
    print(row)
    # break

TITLE                   (Renewed) Aula Enter Tainer SI-9007 Optical Mo...
DESCRIPTION                                                              
BULLET_POINTS           [This Amazon Renewed product will be in an unb...
BRAND                                                                Aula
BROWSE_NODE_ID                                                         98
description_keywords                                                   {}
title_keywords          {tainer, optical, renewed, mouse, enter, black...
Name: 559177, dtype: object


In [ ]:
sub_df["BROWSE_NODE_ID"].values.tolist() * len(all_keywords)

[86, 86, 86, 86]

In [ ]:
# adidas - 10, node - 1
# reebok - 6, node - 1
# reebok - 6, node - 2

In [ ]:
brands_dict = {"brand": [], "keyword": [], "node": []}

for brand in df["BRAND"].unique()[:10]:
  sub_df = df[df["BRAND"] == brand]
  all_keywords = sub_df["description_keywords"].values.flatten()[-1]
  brands_dict["brand"].extend([brand for _ in range(len(all_keywords))])
  brands_dict["keyword"].extend(list(all_keywords))
  brands_dict["node"].extend(sub_df["BROWSE_NODE_ID"].values.tolist())
  if len(brands_dict["brand"]) != len(brands_dict["node"]):
    print(sub_df.shape)
    print(sub_df["BROWSE_NODE_ID"].values.tolist())
  # print(brands_dict)
  # break
# brands = pd.DataFrame(brands_dict)

(2, 7)
[1045, 1045]
(1, 7)
[677]
(1, 7)
[1196]
(3, 7)
[9975, 4418, 1619]
(1, 7)
[3543]
(2, 7)
[2957, 2957]
(2, 7)
[7784, 7784]
(2, 7)
[424886, 709]


In [ ]:
len(brands_dict["brand"]) == len(brands_dict["node"])

True

In [ ]:
brands.tail()

,brand,keyword
131131,RV Enterprises,leftovers
131132,Lifebuoy,hands
131133,Lifebuoy,gentle
131134,Lifebuoy,pack
131135,Lifebuoy,comes


In [ ]:
keyword = brands.groupby(["keyword"])["brand"].apply(list).to_frame()

In [ ]:
keyword["id"] = range(keyword.shape[0])

In [ ]:
keyword.reset_index(level=0, inplace=True)
keyword = keyword[["keyword", "brand"]]

In [ ]:
keyword.head()

,keyword,brand
0,aa,"[Aldo, Fluke, Twiclo, Blue Panda, FAINLIST, BM..."
1,aaa,"[Vaurum, Sri Jagdamba Pearls Dealer, INGLIS LA..."
2,aabattery,[NITPICK]
3,aac,[Everycom]
4,aaditri,[Aaditri Clothing]


In [ ]:
keyword[keyword["keyword"] == "redmi"]["brand"].values

array([list(['RGN', 'estuffz', 'Ragav Enterprise', 'Wolfman', 'A2Z Shop', 'chiraiyaa', 'CRodible Innovation', 'Beyouniq', 'Eglass', 'QRIOH'])],
      dtype=object)